# Aula 14&15

In [2]:
import os 
os.getcwd()
os.chdir('/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_14_15')

In [16]:
!poetry add "uvicorn[standard]"

Using version ^0.29.0 for uvicorn

Updating dependencies
Resolving dependencies... (1.9s)Resolving dependencies... (0.7s)

No dependencies to install or update

Writing lock file


## Criar uma api com fastapi e faker.

In [12]:
%%writefile src/utils/api_produtos.py
from fastapi import FastAPI
from faker import Faker
import pandas as pd
import random

app = FastAPI(debug=True)
fake = Faker()

# Carregar o arquivo CSV de produtos
file_name = 'data/products.csv'
df = pd.read_csv(file_name)
df['indice'] = range(1, len(df) + 1)
df.set_index('indice', inplace=True)

loja_padrao_online = 11

@app.get("/")
async def hello_world():
    """Rota para retornar uma mensagem de saudação."""
    return 'Coca-Cola me patrocina!'

@app.get("/gerar_compra")
async def gerar_compra():
    """Rota para gerar uma compra aleatória."""
    index = random.randint(1, len(df) - 1)
    row = df.iloc[index]
    return [{
        "client": fake.name(),
        "creditcard": fake.credit_card_provider(),
        "product": row["Product Name"],
        "ean": int(row["EAN"]),
        "price": round(float(row["Price"]) * 1.2, 2),
        "clientPosition": fake.location_on_land(),
        "store": loja_padrao_online,
        "dateTime": fake.iso8601()
    }]

@app.get("/gerar_compras/{numero_registro}")
async def gerar_compras(numero_registro: int):
    """Rota para gerar várias compras."""
    if numero_registro < 1:
        return {"error": "O número deve ser maior que 1"}
 
    respostas = []
    for _ in range(numero_registro):
        try:
            index = random.randint(1, len(df) - 1)
            row = df.iloc[index]
            compra = {
                "client": fake.name(),
                "creditcard": fake.credit_card_provider(),
                "product": row["Product Name"],
                "ean": int(row["EAN"]),
                "price": round(float(row["Price"]) * 1.2, 2),
                "clientPosition": fake.location_on_land(),
                "store": loja_padrao_online,
                "dateTime": fake.iso8601()
            }
            respostas.append(compra)
        except Exception as e:
            print(f"Erro inesperado: {e}")
            # Se ocorrer um erro, adiciona uma compra com dados de erro
            compra = {
                "client": fake.name(),
                "creditcard": fake.credit_card_provider(),
                "product": "error",
                "ean": 0,
                "price": 0.0,
                "clientPosition": fake.location_on_land(),
                "store": loja_padrao_online,
                "dateTime": fake.iso8601()
            }
            respostas.append(compra)
    return respostas


Writing src/utils/api_produtos.py


# Documentação API de Geração de Compras

Esta API gera compras aleatórias com base em um conjunto de dados de produtos.

## Rotas Disponíveis

### Rota Principal
- **Descrição**: Retorna uma mensagem de saudação.
- **Endpoint**: `/`
- **Método HTTP**: GET

### Rota de Geração de Compra
- **Descrição**: Gera uma única compra aleatória.
- **Endpoint**: `/gerar_compra`
- **Método HTTP**: GET

### Rota de Geração de Compras
- **Descrição**: Gera um número específico de compras aleatórias.
- **Endpoint**: `/gerar_compras/{numero_registro}`
- **Parâmetro de Path**: `numero_registro` (int) - Número de compras a serem geradas.
- **Método HTTP**: GET

## Detalhes das Respostas

### Resposta da Rota de Geração de Compra

- **Formato JSON**:

```json
{
    "client": "Nome do Cliente",
    "creditcard": "Provedor do Cartão de Crédito",
    "product": "Nome do Produto",
    "ean": "Código EAN",
    "price": "Preço",
    "clientPosition": "Localização do Cliente",
    "store": "ID da Loja",
    "dateTime": "Data e Hora da Compra"
}


## Criando uma lista de 200 produtos com faker.

In [11]:
from faker import Faker
import pandas as pd

# Criar um objeto Faker com localização em português brasileiro
fake = Faker('pt_BR')

# Lista de palavras-chave para produtos eletrônicos
keywords = ['smartphone', 'tablet', 'notebook', 'smartwatch', 'câmera', 'fones de ouvido', 'TV', 'console de videogame', 'impressora']

# Lista de produtos eletrônicos em português brasileiro
electronic_products = [f'{fake.random.choice(keywords)}' for _ in range(200)]

# Lista de códigos EAN (European Article Number)
eans = [fake.ean() for _ in range(200)]

# Lista de preços
prices = [round(fake.pyfloat(min_value=1, max_value=100), 2) for _ in range(200)]

# Criar DataFrame
data = {
    "Product Name": electronic_products,
    "EAN": eans,
    "Price": prices
}
df = pd.DataFrame(data)

# Salvar DataFrame como arquivo CSV
df.to_csv("data/products.csv", index=False)


### Executando a api:

In [17]:
!uvicorn src.utils.api_produtos:app --reload

INFO:     Will watch for changes in these directories: ['/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_14_15']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [67910] using WatchFiles
INFO:     Started server process [67913]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:40904 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:40904 - "GET /github-markdown-css HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:40916 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:40916 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:40916 - "GET /github-markdown-css HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57188 - "GET /gerar_compras/20 HTTP/1.1" 200 OK
INFO:     127.0.0.1:48016 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:48016 - "GET /github-markdown-css HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:48016 - "GET /gerar_compra HTTP/1.1" 200 OK
INFO:     127.0.0.1:48016 - "